In [1]:
import pandas as pd
import numpy as np
import sys
import os
os.chdir('/Users/veronica/Documents/GitHub/analysis-tools')
from src import utils, credentials

In [2]:
creds = credentials.creds
engine = utils.create_sql_db(creds)

In [3]:
query = """WITH base as (
SELECT 
  c.contract_id,
  c.store_id,
  c.seller_name,
  c.plan_id,
  c.plan_purchase_price,
  pcrs_contracts.retail_cost_of_plan,
  pcrs_contracts.product_purchase_price,
  c.product_list_price,
  c.plan_sku,
  c.status,
  prod.reimbursement_amount/100.0 as reimbursement_amount,
  prod.price as price,
  CASE WHEN prod.reimbursement_amount IS NOT NULL THEN prod.reimbursement_amount/100.0 ELSE prod.price END as price_final,
  m.merchant_cut,
  p.retail_target,
  p.fixed_price,
  p.price_high,
  p.price_low
FROM analytics.contract_facts c
LEFT JOIN analytics.merchant_facts m -- get merchant_cut
ON c.store_id = m.store_id
LEFT JOIN analytics.product_facts prod -- get reimbursement amount
ON c.product_id = prod.product_id AND c.store_id = prod.store_id
LEFT JOIN analytics_pcrs.pcrs_contracts_sold AS pcrs_contracts
ON c.admin_contract_id = pcrs_contracts.contract_number
LEFT JOIN analytics.plans_pricing p
ON c.plan_id = p.plan_id
WHERE prod.price BETWEEN (p.price_low/100.0) and (p.price_high/100.0) 
AND c.status in ('expired', 'live')
)

SELECT
  b.*,
  pm.cost/100.0 as cost
FROM base b
LEFT JOIN analytics.plans_matching pm -- get cost (premium) based on price_final (reimbursement_amount/product_list_price)
ON b.plan_id = pm.plan_id AND b.plan_sku = pm.vendor_sku AND b.price_high = pm.price_high and b.price_low = pm.price_low
WHERE 
b.price_final BETWEEN (b.price_low/100.0) AND (b.price_high/100.0)"""

df = pd.read_sql_query(query, engine)

In [11]:
# Clean Data
# Remove contracts with a fixed price, empty cost value, where cost=1, or product_price=1

In [4]:
df_clean = df[(df['cost'].notnull()) & (df['cost']!= 0.01) & 
              (df['fixed_price'].isnull()) & (df['price']!= 1)]
df_clean

,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,status,reimbursement_amount,price,price_final,merchant_cut,retail_target,fixed_price,price_high,price_low,cost
0,f1ce4865-17ac-4a4d-a39b-4b315b959804,a5e80a29-c6af-446f-b569-f021e697c48c,YI,10001-misc-elec-base-replace-3y,23.99,23.99,74.99,74.99,EXTRPL37,live,89.99,89.99,89.99,0.25,0.260,NaN,9999,5000,11.16
3,de6851c5-fc89-439f-8c9c-916e6b294c50,446af1d1-22e5-41cb-bbb1-dc6e7ad970b4,1MOREUSA,10001-misc-elec-base-replace-1y,10.99,10.99,99.99,99.99,EXTRPL3,live,69.99,69.99,69.99,0.25,0.100,NaN,9999,5000,3.85
6,d235b646-3cfe-4051-811a-9711a6ceb8d7,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-3y,20.99,20.99,79.99,79.99,EXTRPL37,live,79.99,79.99,79.99,0.30,0.260,NaN,9999,5000,11.16
13,56376ea1-b6c1-49a2-a895-3743b8cc73ce,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-2y,44.99,44.99,329.99,329.99,EXTRPL25,live,349.99,349.99,349.99,0.35,0.180,NaN,34999,30000,26.49
14,a53353c5-1e55-426a-8541-a15c89da3b68,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-2y,42.99,42.99,299.95,299.95,EXTRPL24,live,299.95,299.95,299.95,0.35,0.180,NaN,29999,25000,22.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365741,01d1b813-84df-45b0-9f27-63e15a2a5c19,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-office-equip-adh-replace-3y,26.99,NaN,NaN,74.99,EXTOE129,live,74.99,74.99,74.99,0.50,0.290,NaN,7499,5000,10.40
365742,3303b054-714a-4666-8e6d-195638733feb,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-office-equip-adh-replace-1y,11.99,NaN,NaN,99.99,EXTOE88,live,99.99,99.99,99.99,0.50,0.115,NaN,9999,7500,4.73
365747,e04e8e1c-0c27-4dfb-bc6c-d8a29e4bc56f,c41e5a0d-e5f8-4761-8132-9d48fbc1067b,Logitech,A0-ELPAU-2y,48.99,NaN,NaN,179.99,EXTPORAUD24,live,179.99,179.99,179.99,0.30,0.270,NaN,19999,15000,18.32
365750,dcd0cd1a-68a9-4e39-90b1-4a130e8912a4,2c4c3873-0268-4440-bcd4-3cb383450c31,PCLiquidations,10001-misc-elec-adh-replace-1y,3.09,NaN,NaN,14.99,EXTRPL52,live,14.99,14.99,14.99,0.50,0.100,NaN,2499,0,1.23


In [5]:
def calc_target_price(df):
    df['target_price'] = df['retail_target'] * df['price']
    return df

In [6]:
def calc_min_price(df, extend_cut):
    df[f'min_price_{extend_cut}'] = df['cost'] / (1 - df['merchant_cut'] - extend_cut)
    return df

In [7]:
def calc_diff(df, min_price_col):
    df['price_diff'] = df['target_price'] - df[min_price_col]
    df['price_diff_pct'] = (df['target_price'] - df[min_price_col])/ df[min_price_col]
    return df

In [8]:
def find_price_floor(df, min_price_col):
    df['price_floor'] = np.where(df[min_price_col] > df['target_price'], 1, 0)
    return df

In [9]:
df_price = calc_target_price(df_clean)
df_price = calc_min_price(df_price, 0.10)
df_price = calc_diff(df_price, 'min_price_0.1')
df_price = find_price_floor(df_price, 'min_price_0.1')
df_price.head()

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,contract_id,store_id,seller_name,plan_id,plan_purchase_price,retail_cost_of_plan,product_purchase_price,product_list_price,plan_sku,status,...,retail_target,fixed_price,price_high,price_low,cost,target_price,min_price_0.1,price_diff,price_diff_pct,price_floor
0,f1ce4865-17ac-4a4d-a39b-4b315b959804,a5e80a29-c6af-446f-b569-f021e697c48c,YI,10001-misc-elec-base-replace-3y,23.99,23.99,74.99,74.99,EXTRPL37,live,...,0.26,NaN,9999,5000,11.16,23.3974,17.169231,6.228169,0.362752,0
3,de6851c5-fc89-439f-8c9c-916e6b294c50,446af1d1-22e5-41cb-bbb1-dc6e7ad970b4,1MOREUSA,10001-misc-elec-base-replace-1y,10.99,10.99,99.99,99.99,EXTRPL3,live,...,0.10,NaN,9999,5000,3.85,6.9990,5.923077,1.075923,0.181649,0
6,d235b646-3cfe-4051-811a-9711a6ceb8d7,2798ad2f-a9cb-4ed0-b787-30ea6b2dfc2b,August Home,10001-misc-elec-base-replace-3y,20.99,20.99,79.99,79.99,EXTRPL37,live,...,0.26,NaN,9999,5000,11.16,20.7974,18.600000,2.197400,0.118140,0
13,56376ea1-b6c1-49a2-a895-3743b8cc73ce,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-2y,44.99,44.99,329.99,329.99,EXTRPL25,live,...,0.18,NaN,34999,30000,26.49,62.9982,48.163636,14.834564,0.308003,0
14,a53353c5-1e55-426a-8541-a15c89da3b68,2e1df809-4f73-4943-9bde-49fdbeddb8fc,NewAir,10001-misc-elec-base-replace-2y,42.99,42.99,299.95,299.95,EXTRPL24,live,...,0.18,NaN,29999,25000,22.40,53.9910,40.727273,13.263727,0.325672,0


In [10]:
len(df_price)

183974

In [11]:
df_pf = df_price[df_price['price_floor']==1]

In [12]:
len(df_pf)

62258

In [13]:
len(df_pf['store_id'].unique())

159

In [14]:
df_pf['seller_name'].value_counts().head(20)

BlendJet                  16284
JBL-US                    10152
RealTruck                  3933
L'ange Hair                3881
Carandtruckremotes.com     3837
Skullcandy                 2124
EightVape                  1955
DailySale                  1541
RaceDayQuads               1262
Parts Dr                   1229
1MOREUSA                   1078
Jetson Electric Bikes      1069
Logitech                   1025
Motosport                   858
Jegs High Performance       765
Wyze Ecommerce              684
PCLiquidations              648
AutoAnything                606
Blackstone Products         604
Backcountry.com             584
Name: seller_name, dtype: int64

In [18]:
# diff between plan purchase price and minimum price
df_pf['retail_min_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['min_price_0.1'])
# min price rounded to the nearest dollar
df_pf['min_price_int'] = np.ceil(df_pf['min_price_0.1'])

# diff between plan purchase price and target price
df_pf['retail_target_diff'] = abs(df_pf['retail_cost_of_plan'] - df_pf['target_price'])

# diff between minimum price and target price
df_pf['min_target_diff'] = abs(df_pf['min_price_0.1'] - df_pf['target_price'])

# ratio of min price to product price
df_pf['min_target'] = df_pf['min_price_0.1']/df_pf['price']

# diff between (min price/product price) and retail target
df_pf['target_diff'] = df_pf['min_target'] - df_pf['retail_target']

/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Users/veronica/env/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [19]:
df_pf['seller_name'].value_counts().head(10)

BlendJet                  16284
JBL-US                    10152
RealTruck                  3933
L'ange Hair                3881
Carandtruckremotes.com     3837
Skullcandy                 2124
EightVape                  1955
DailySale                  1541
RaceDayQuads               1262
Parts Dr                   1229
Name: seller_name, dtype: int64

In [42]:
col_list = ['product_list_price', 'price', 'plan_purchase_price', 'retail_cost_of_plan', 
            'target_price', 'min_price_0.1',  'retail_min_diff', 'min_target', 'retail_target',
           'target_diff', 'min_target_diff']

In [43]:
df_diff = df_pf[df_pf['retail_cost_of_plan'] < df_pf['min_price_0.1']].sort_values(by='retail_min_diff', ascending=False)
df_diff[col_list]

,product_list_price,price,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff,min_target,retail_target,target_diff,min_target_diff
101769,2344.99,2344.99,649.00,649.00,609.6974,883.366667,234.366667,0.376704,0.26,0.116704,273.669267
126947,2344.99,2344.99,649.00,649.00,609.6974,883.366667,234.366667,0.376704,0.26,0.116704,273.669267
43686,7299.99,7599.99,949.00,949.00,987.9987,1120.200000,171.200000,0.147395,0.13,0.017395,132.201300
130774,1995.00,1995.00,549.00,549.00,518.7000,688.433333,139.433333,0.345079,0.26,0.085079,169.733333
45280,8199.99,8499.99,1099.00,1099.00,1104.9987,1227.700000,128.700000,0.144435,0.13,0.014435,122.701300
...,...,...,...,...,...,...,...,...,...,...,...
270202,6.95,6.95,1.19,1.19,1.1815,1.200000,0.010000,0.172662,0.17,0.002662,0.018500
333560,59.99,59.99,6.49,6.49,5.9990,6.500000,0.010000,0.108351,0.10,0.008351,0.501000
228669,59.99,59.99,6.49,6.49,5.9990,6.500000,0.010000,0.108351,0.10,0.008351,0.501000
337330,59.99,59.99,6.49,6.49,5.9990,6.500000,0.010000,0.108351,0.10,0.008351,0.501000


In [33]:
df_diff['retail_min_diff'].describe()

count    19548.000000
mean         1.692383
std          7.180902
min          0.010000
25%          0.310000
50%          0.310000
75%          0.310000
max        234.366667
Name: retail_min_diff, dtype: float64

In [75]:
df_anom = df_pf_clean[(df_pf_clean['retail_target_diff'] < df_pf_clean['retail_min_diff'])].sort_values(by='retail_min_diff')

df_anom[col_list]

,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,product_list_price,price_final,retail_target,min_target,target_diff
279876,1.19,1.19,1.1815,1.200000,6.95,6.95,0.17,0.172662,0.002662
226229,1.19,1.19,1.1815,1.200000,6.95,6.95,0.17,0.172662,0.002662
354407,1.19,1.19,1.1815,1.200000,6.95,6.95,0.17,0.172662,0.002662
21554,1.59,1.59,1.5735,1.660000,10.49,10.49,0.15,0.158246,0.008246
275206,2.29,2.29,2.2570,2.366667,22.57,22.57,0.10,0.104859,0.004859
...,...,...,...,...,...,...,...,...,...
208268,1099.00,1099.00,1104.9987,1227.700000,8199.99,8499.99,0.13,0.149720,0.019720
104199,549.00,549.00,518.7000,688.433333,1995.00,1995.00,0.26,0.345079,0.085079
38535,949.00,949.00,987.9987,1120.200000,7299.99,7599.99,0.13,0.153452,0.023452
83053,649.00,649.00,609.6974,883.366667,2344.99,2344.99,0.26,0.376704,0.116704


In [65]:
df_anom['seller_name'].value_counts().head(20)

BlendJet                  16283
DailySale                   940
AutoAnything                482
Motosport                   312
JBL-US                      215
Backcountry.com             127
Shop Indoor Golf             96
MoissaniteCo                 79
Competitive Cyclist          69
Zebit                        56
August Home                  42
Jegs High Performance        33
NewAir                       22
RealTruck                    22
PADO                         16
Smoke Cartel                 15
White Duck Outdoors          10
Evolve Skateboards USA        8
Active Lifestyle Store        7
New Wave Toys                 4
Name: seller_name, dtype: int64

In [ ]:
rounding_band = [[0.09, 4.99, 0.1], [5.49, 9.99, 0.5], 
                [10.99, 49.99, 1], ]

In [22]:
test = [5.49, 9.99, 0.5]
np.arange(test[0], test[1], test[2])

array([5.49, 5.99, 6.49, 6.99, 7.49, 7.99, 8.49, 8.99, 9.49])

## Impact Analysis
- how large is the difference between min_price and target_price?
- which merchants are most impacted?

In [34]:
df_pf['seller_name'].value_counts()

BlendJet                                              16284
JBL-US                                                10152
RealTruck                                              3933
L'ange Hair                                            3881
Carandtruckremotes.com                                 3837
                                                      ...  
Direct Fire Pits                                          1
GoGoPowerJuice                                            1
Lumicharge- Stylish LED Lamp ,Phone Charger & More        1
Jamstack                                                  1
TimTam                                                    1
Name: seller_name, Length: 159, dtype: int64

In [55]:
df_pf_filt = df_pf[df_pf['retail_min_diff']<5].sort_values(by='retail_min_diff', ascending=False)
df_pf_filt[col_list]

,product_list_price,price,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff,min_target,retail_target,target_diff,min_target_diff
324484,59.95,59.95,7.49,7.49,7.1940,12.466667,4.976667,0.207951,0.12,0.087951,5.272667
231770,235.00,235.00,54.99,54.99,47.0000,50.025000,4.965000,0.212872,0.20,0.012872,3.025000
241736,235.00,235.00,54.99,54.99,47.0000,50.025000,4.965000,0.212872,0.20,0.012872,3.025000
314658,235.00,235.00,54.99,54.99,47.0000,50.025000,4.965000,0.212872,0.20,0.012872,3.025000
288871,230.00,230.00,54.99,54.99,46.0000,50.025000,4.965000,0.217500,0.20,0.017500,4.025000
...,...,...,...,...,...,...,...,...,...,...,...
259774,19.75,19.95,1.99,1.99,1.7955,1.984615,0.005385,0.099479,0.09,0.009479,0.189115
341081,29.00,29.00,23.99,23.99,9.2800,23.984615,0.005385,0.827056,0.32,0.507056,14.704615
356512,29.00,29.00,23.99,23.99,9.2800,23.984615,0.005385,0.827056,0.32,0.507056,14.704615
91415,69.00,69.00,23.99,23.99,22.0800,23.984615,0.005385,0.347603,0.32,0.027603,1.904615


In [78]:
df_pf['seller_name'].value_counts().head(25)

BlendJet                  16284
JBL-US                    10152
RealTruck                  3933
L'ange Hair                3881
Carandtruckremotes.com     3837
Skullcandy                 2124
EightVape                  1955
DailySale                  1541
RaceDayQuads               1262
Parts Dr                   1229
1MOREUSA                   1078
Jetson Electric Bikes      1069
Logitech                   1025
Motosport                   858
Jegs High Performance       765
Wyze Ecommerce              684
PCLiquidations              648
AutoAnything                606
Blackstone Products         604
Backcountry.com             584
Renogy                      556
August Home                 544
T3                          495
VORO MOTORS, Inc.           452
Zebit                       369
Name: seller_name, dtype: int64

In [62]:
df_pf['min_target_diff'].describe()

count    62258.000000
mean         5.560582
std         18.884403
min          0.000400
25%          0.384900
50%          0.976923
75%          3.861000
max        412.450000
Name: min_target_diff, dtype: float64

In [64]:
df_pf[df_pf['min_target_diff'] > 3.86]['seller_name'].value_counts()

JBL-US                   4596
RealTruck                3284
Jetson Electric Bikes     926
DailySale                 777
Skullcandy                695
                         ... 
Top Tech Electronics        1
Direct Fire Pits            1
OEM Car Key Mall            1
Rocksteady Audio            1
TruWood                     1
Name: seller_name, Length: 95, dtype: int64

In [ ]:
df_diff_high = df_pf[df_pf['min_target_diff'] > 3.86]
df_diff_high['min_target_diff'].hist()

In [71]:
df_pf[df_pf['min_target_diff'] <= 5]['seller_name'].value_counts()

BlendJet                                              16284
JBL-US                                                 6466
L'ange Hair                                            3863
Carandtruckremotes.com                                 3802
EightVape                                              1948
                                                      ...  
Lumicharge- Stylish LED Lamp ,Phone Charger & More        1
Jamstack                                                  1
Tentsile                                                  1
JDMuscle                                                  1
GoGoPowerJuice                                            1
Name: seller_name, Length: 140, dtype: int64

In [73]:
df_pf[(df_pf['min_target_diff'] > 5) & (df_pf['min_target_diff'] <= 15)]['seller_name'].value_counts()

RealTruck              2873
JBL-US                 2486
Skullcandy              298
Blackstone Products     289
DailySale               283
                       ... 
QuietKat E-Bikes          1
Morris 4x4                1
ZVOX Audio                1
Pool Warehouse            1
MDMaxx                    1
Name: seller_name, Length: 64, dtype: int64

In [83]:
df_pf[(df_pf['min_target_diff'] > 15) & (df_pf['min_target_diff'] <= 50)]['seller_name'].value_counts().head(20)

JBL-US                   1194
Jetson Electric Bikes     602
DailySale                 387
AutoAnything              100
RealTruck                  94
Blackstone Products        72
Competitive Cyclist        70
Backcountry.com            67
Motosport                  55
HA-US                      54
MoissaniteCo               51
Med Mart                   48
Shop Indoor Golf           45
Revel Boards               42
Jegs High Performance      38
Auto Rim Shop              34
Home Shopping Malls        28
ds18caraudio               27
Crown & Caliber            26
throtl                     21
Name: seller_name, dtype: int64

In [84]:
df_pf[df_pf['min_target_diff'] > 50]['seller_name'].value_counts().head(20)

VORO MOTORS, Inc.           451
Jetson Electric Bikes       159
Brilliant Earth             148
Shop Indoor Golf             57
REVRides                     50
MoissaniteCo                 45
EcoReco Electric Scooter     34
Med Mart                     31
Revel Boards                 27
EUCO                         21
DailySale                    16
Blackstone Products          14
Backcountry.com              14
Competitive Cyclist          13
Motosport                    10
RealTruck                     9
AutoAnything                  9
JBL-US                        6
Prima Coffee                  6
MiniMotors USA                5
Name: seller_name, dtype: int64

In [82]:
df_pf[df_pf['seller_name']=='VORO MOTORS, Inc.'][col_list].sort_values(by='min_target_diff', ascending=False)

,product_list_price,price,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff,min_target,retail_target,target_diff,min_target_diff
192586,2495.0,2995.0,999.0,999.0,539.10,951.550000,47.450000,0.317713,0.18,0.137713,412.450000
106502,2499.0,2995.0,999.0,999.0,539.10,951.550000,47.450000,0.317713,0.18,0.137713,412.450000
241252,2999.0,2999.0,999.0,999.0,539.82,951.550000,47.450000,0.317289,0.18,0.137289,411.730000
212767,2799.0,2999.0,999.0,999.0,539.82,951.550000,47.450000,0.317289,0.18,0.137289,411.730000
234191,2199.0,2199.0,299.0,799.0,395.82,778.566667,20.433333,0.354055,0.18,0.174055,382.746667
...,...,...,...,...,...,...,...,...,...,...,...
353979,899.0,899.0,149.0,149.0,89.90,146.966667,2.033333,0.163478,0.10,0.063478,57.066667
354392,899.0,899.0,149.0,149.0,89.90,146.966667,2.033333,0.163478,0.10,0.063478,57.066667
270431,899.0,899.0,149.0,149.0,89.90,146.966667,2.033333,0.163478,0.10,0.063478,57.066667
179904,899.0,899.0,149.0,149.0,89.90,146.966667,2.033333,0.163478,0.10,0.063478,57.066667


In [85]:
df_pf[df_pf['seller_name']=='Jetson Electric Bikes'][col_list].sort_values(by='min_target_diff', ascending=False)

,product_list_price,price,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff,min_target,retail_target,target_diff,min_target_diff
302251,899.99,899.99,329.00,329.00,161.9982,320.654545,8.345455,0.356287,0.18,0.176287,158.656345
175552,899.99,899.99,329.00,329.00,161.9982,320.654545,8.345455,0.356287,0.18,0.176287,158.656345
20663,519.99,599.99,229.00,229.00,83.9986,226.109091,2.890909,0.376855,0.14,0.236855,142.110491
12316,599.99,599.99,229.00,229.00,83.9986,226.109091,2.890909,0.376855,0.14,0.236855,142.110491
297950,799.99,799.99,249.00,249.00,111.9986,240.509091,8.490909,0.300640,0.14,0.160640,128.510491
...,...,...,...,...,...,...,...,...,...,...,...
177159,69.99,69.99,7.49,7.49,6.9990,7.000000,0.490000,0.100014,0.10,0.000014,0.001000
251277,59.99,69.99,7.49,7.49,6.9990,7.000000,0.490000,0.100014,0.10,0.000014,0.001000
75215,69.99,69.99,7.49,7.49,6.9990,7.000000,0.490000,0.100014,0.10,0.000014,0.001000
100755,69.99,69.99,7.49,7.49,6.9990,7.000000,0.490000,0.100014,0.10,0.000014,0.001000


In [86]:
df_pf[df_pf['seller_name']=='Brilliant Earth'][col_list].sort_values(by='min_target_diff', ascending=False)

,product_list_price,price,plan_purchase_price,retail_cost_of_plan,target_price,min_price_0.1,retail_min_diff,min_target,retail_target,target_diff,min_target_diff
358369,2650.0,2650.0,499.00,499.00,212.00,492.12,6.88,0.185706,0.080,0.105706,280.12
355268,2950.0,2950.0,1499.00,499.00,236.00,492.12,6.88,0.166820,0.080,0.086820,256.12
348604,2990.0,2990.0,599.00,499.00,239.20,492.12,6.88,0.164589,0.080,0.084589,252.92
345665,3000.0,3000.0,499.00,499.00,240.00,492.12,6.88,0.164040,0.080,0.084040,252.12
169063,3090.0,3090.0,1499.00,499.00,247.20,492.12,6.88,0.159262,0.080,0.079262,244.92
...,...,...,...,...,...,...,...,...,...,...,...
365668,1190.0,1190.0,139.00,NaN,71.40,72.76,NaN,0.061143,0.060,0.001143,1.36
348451,250.0,250.0,26.99,26.99,25.00,26.16,0.83,0.104640,0.100,0.004640,1.16
362980,1590.0,1590.0,94.99,NaN,87.45,88.60,NaN,0.055723,0.055,0.000723,1.15
334684,1590.0,1590.0,139.00,89.99,87.45,88.60,1.39,0.055723,0.055,0.000723,1.15
